In [2]:
import pandas as pd
import json

df = pd.read_csv("/ltstorage/home/strich/STASC/logs/test_data_qwen_2.5_1.5b_hotpot_baseline_cot.csv").head(10)
df["initial_generation"][0]  # Display the first initial generation

'["Step 1: Identify the foreign born victim of Singapore\'s caning punishment.\\nThe foreign born victim was Oliver Fricker.\\n\\nStep 2: Determine Oliver Fricker\'s nationality.\\nOliver Fricker is British.\\n\\nStep 3: Confirm that Oliver Fricker was a victim of Singapore\'s caning punishment.\\nOliver Fricker was indeed a victim of Singapore\'s caning punishment.\\n\\nStep 4: Conclude with the final answer.\\nThe nationality of the foreign born victim of Singapore\'s caning punishment before Oliver Fricker experienced the same was British."]'

In [7]:
with open("/ltstorage/home/strich/STASC/outputs/25-08-11_18:41_qwen_2.5_1.5b/inference_log.json", "r") as f:
    data = json.load(f)

df_json = pd.DataFrame(data)
df_json["response"][0]  # Display the first response from the JSON data

"step 1: identify the context of the question - the question is about the nationality of a foreign-born victim of singapore's caning punishment.\nstep 2: determine the nationality of the victim - the victim was oliver fricker, a british national.\nstep 3: confirm the nationality of the foreign-born victim - the foreign-born victim was also oliver fricker, a british national.\nstep 4: conclude with the final answer - the nationality of the foreign-born victim of singapore's caning punishment before oliver fricker experienced the same was british.\nfinal answer: british"

In [24]:
from rouge_score import rouge_scorer

root = "/ltstorage/home/strich/STASC/"
folder_1 = "outputs/25-08-12_12:40_qwen_2.5_1.5b"
folder_2 = "outputs/25-08-12_12:43_qwen_2.5_1.5b"

with open(f"{root}/{folder_1}/inference_log.json", "r") as f:
    data = json.load(f)
df_json = pd.DataFrame(data)

with open(f"{root}/{folder_2}/inference_log.json", "r") as f:
    data = json.load(f)
df_json_2 = pd.DataFrame(data)

merged = df_json.merge(
    df_json_2, on=["user_prompt"], how="left", suffixes=("_1", "_2")
)
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_scores = merged.apply(
    lambda row: scorer.score(str(row["response_1"]), str(row["response_2"]))["rougeL"].fmeasure,
    axis=1
)
merged["rougeL_f1"] = rouge_scores
df_filtered = merged[merged["response_1"] != merged["response_2"]]
df_filtered[["user_prompt", "response_1", "response_2", "rougeL_f1"]]


,user_prompt,response_1,response_2,rougeL_f1
